In [19]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))


In [ ]:
download_file('/kaggle/working/submission.csv', 'out')


In [5]:
import pandas as pd
from datasets import load_dataset, Audio
from pathlib import Path

train_df = pd.read_csv("/kaggle/input/shl-dataset/dataset/train.csv")
train_df['path'] = train_df['filename'].apply(lambda x: f"/kaggle/input/shl-dataset/dataset/audios_train/{x}")

In [6]:
import torch
import librosa
from transformers import Wav2Vec2Processor, Wav2Vec2Model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h").to(device)

def extract_features(path):
    speech, sr = librosa.load(path, sr=16000)
    inputs = processor(speech, sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        outputs = model(inputs.input_values.to(device)).last_hidden_state
    # Average pooling across time axis
    return outputs.mean(dim=1).squeeze().cpu().numpy()

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from tqdm import tqdm
import numpy as np

X = []
for path in tqdm(train_df['path']):
    X.append(extract_features(path))

X = np.array(X)
y = train_df['label'].values

100%|██████████| 444/444 [05:34<00:00,  1.33it/s]


In [85]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [71]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

xgb = XGBRegressor(n_estimators=1500, max_depth=6, learning_rate=0.4, random_state=42)
xgb.fit(X_train, y_train)

preds = xgb.predict(X_val)
rmse = mean_squared_error(y_val, preds, squared=False)
print("XGBoost RMSE:", rmse)

XGBoost RMSE: 1.0576652404800053


In [10]:
from sklearn.svm import SVR

svr = SVR(C=1.0, epsilon=0.1)
svr.fit(X_train, y_train)

preds = svr.predict(X_val)
rmse = mean_squared_error(y_val, preds, squared=False)
print("SVR RMSE:", rmse)

SVR RMSE: 1.06141962230121


In [27]:
test_df = pd.read_csv("/kaggle/input/shl-dataset/dataset/test.csv")
test_df['path'] = test_df['filename'].apply(lambda x: f"/kaggle/input/shl-dataset/dataset/audios_test/{x}")

In [32]:
X_test = []
for path in tqdm(test_df['path']):
    X_test.append(extract_features(path))

X_test = np.array(X_test)

100%|██████████| 195/195 [01:56<00:00,  1.68it/s]


In [72]:
test_preds = xgb.predict(X_test)  # or svr.predict(X_test)

In [73]:
submission_df = pd.DataFrame({
    "filename": test_df["filename"],
    "label": test_preds
})
submission_df.to_csv("submission.csv", index=False)

print("✅ submission.csv has been saved successfully.")

✅ submission.csv has been saved successfully.


In [74]:
download_file('submission.csv', 'out')


/kaggle/working/out.zip

In [23]:
test_preds2=svr.predict(X_test)

In [86]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Train XGBoost regressor
model_xgb = XGBRegressor(n_estimators=1500, max_depth=5, learning_rate=0.1,random_state=42)
model_xgb.fit(X_train, y_train)

# Predict on validation
val_preds = model_xgb.predict(X_val)
val_preds = np.clip(val_preds, 0, 5)  # Clipping
val_preds = 0.8 * val_preds + 0.2 * y_train.mean()  # Smoothing
print("Validation RMSE:", mean_squared_error(y_val, val_preds, squared=False))

Validation RMSE: 0.938543161067362


In [87]:
# Predict on test set
test_preds = model_xgb.predict(X_test)
test_preds = np.clip(test_preds, 0, 5)
test_preds = 0.9 * test_preds + 0.1 * y_train.mean()  # Smoothing

# Save to submission.csv
submission = pd.DataFrame({
    "filename": test_df["filename"],
    "label": test_preds
})
submission.to_csv("submission.csv", index=False)
print("Saved to submission.csv ✅")

Saved to submission.csv ✅


In [88]:
download_file('submission.csv', 'out')


/kaggle/working/out.zip

In [108]:
pip install openai-whisper librosa sentence-transformers xgboost

Note: you may need to restart the kernel to use updated packages.


In [109]:
import os
import librosa
import numpy as np
import whisper
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sentence_transformers import SentenceTransformer
from xgboost import XGBRegressor

# Load Data
train_df = pd.read_csv("/kaggle/input/shl-dataset/dataset/train.csv")
test_df = pd.read_csv("/kaggle/input/shl-dataset/dataset/test.csv")

# Load Whisper model (heavier one)
whisper_model = whisper.load_model("medium")

# Text encoder
text_encoder = SentenceTransformer("all-mpnet-base-v2")

def extract_audio_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    zcr = librosa.feature.zero_crossing_rate(y)[0]
    rms = librosa.feature.rms(y=y)[0]
    return np.concatenate([
        np.mean(mfccs, axis=1),
        np.std(mfccs, axis=1),
        [np.mean(zcr), np.std(zcr)],
        [np.mean(rms), np.std(rms)]
    ])

# Transcribe + extract features
def extract_features(df, audio_folder):
    text_embeddings, audio_features = [], []

    for file in tqdm(df['filename']):
        file_path = os.path.join(audio_folder, file)

        # Transcription
        result = whisper_model.transcribe(file_path, fp16=False)
        transcript = result['text']

        # Text embedding
        text_embed = text_encoder.encode(transcript)

        # Audio features
        audio_feat = extract_audio_features(file_path)

        text_embeddings.append(text_embed)
        audio_features.append(audio_feat)

    text_embeddings = np.array(text_embeddings)
    audio_features = np.array(audio_features)

    # Concatenate
    combined_features = np.concatenate([text_embeddings, audio_features], axis=1)
    return combined_features

# Extract training features
train_features = extract_features(train_df, "/kaggle/input/shl-dataset/dataset/audios_train")
train_labels = train_df['label'].values

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(train_features, train_labels, test_size=0.1, random_state=42)

# Train XGBoost
model = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.01,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model.fit(X_train, y_train, early_stopping_rounds=20, eval_set=[(X_val, y_val)], verbose=False)

# Validate
val_preds = model.predict(X_val)
rmse = mean_squared_error(y_val, val_preds, squared=False)
print(f"Validation RMSE: {rmse:.4f}")

100%|█████████████████████████████████████| 1.42G/1.42G [00:37<00:00, 41.3MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|█████████▉| 443/444 [1:18:26<00:08,  8.63s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 444/444 [1:18:33<00:00, 10.61s/it]
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Validation RMSE: 0.8779


In [112]:
# Test set
test_features = extract_features(test_df, "/kaggle/input/shl-dataset/dataset/audios_test")

# Predict
test_preds = model.predict(test_features)

# Clip or smooth if needed
test_preds = np.clip(test_preds, 0, 5)

# Save submission
submission_df = pd.DataFrame({
    "filename": test_df["filename"],
    "label": test_preds
})
submission_df.to_csv("submission.csv", index=False)
print("submission.csv saved!")

  0%|          | 0/195 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  1%|          | 1/195 [01:06<3:33:26, 66.01s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  1%|          | 2/195 [01:10<1:35:39, 29.74s/it]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 99%|█████████▉| 194/195 [33:16<00:38, 38.08s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 195/195 [33:50<00:00, 10.41s/it]

submission.csv saved!


In [113]:
download_file('submission.csv', 'out')


/kaggle/working/out.zip

In [177]:
X_train, X_val, y_train, y_val = train_test_split(train_features, train_labels, test_size=0.3, random_state=42)

# Train XGBoost
model = XGBRegressor(
    n_estimators=400,
    learning_rate=0.009,
    max_depth=6,
    random_state=42
)
model.fit(X_train, y_train,  eval_set=[(X_val, y_val)], verbose=False)

# Validate
val_preds = model.predict(X_val)
rmse = mean_squared_error(y_val, val_preds, squared=False)
print(f"Validation RMSE: {rmse:.4f}")

Validation RMSE: 0.9404


In [178]:
test_preds = model.predict(test_features)



# Save submission
submission_df = pd.DataFrame({
    "filename": test_df["filename"],
    "label": test_preds
})
submission_df.to_csv("submission.csv", index=False)
print("submission.csv saved!")

submission.csv saved!


In [179]:
download_file('submission.csv', 'out')


/kaggle/working/out.zip

In [262]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor(hidden_layer_sizes=(128,120,100,80,40,20,10,5), activation='relu', solver='adam', max_iter=100000, random_state=42)
model.fit(X_train, y_train)

val_preds = model.predict(X_val)
rmse = mean_squared_error(y_val, val_preds, squared=False)
print(f"Validation RMSE (MLP Neural Net): {rmse:.4f}")

Validation RMSE (MLP Neural Net): 0.8968


In [263]:
test_preds = model.predict(test_features)



# Save submission
submission_df = pd.DataFrame({
    "filename": test_df["filename"],
    "label": test_preds
})
submission_df.to_csv("submission.csv", index=False)
print("submission.csv saved!")

submission.csv saved!


In [264]:
download_file('submission.csv', 'out')


/kaggle/working/out.zip

In [279]:
def extract_whisper_embedding(audio_path):
    audio = whisper.load_audio(audio_path)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(device)

    with torch.no_grad():
        encoded = model.encoder(mel.unsqueeze(0))
    
    return encoded.squeeze(0).mean(dim=0).cpu().numpy()

In [271]:
train_df = pd.read_csv("/kaggle/input/shl-dataset/dataset/train.csv")
train_df['file_path'] = train_df['filename'].apply(lambda x: os.path.join("/kaggle/input/shl-dataset/dataset/audios_train", x))

X = []
y = []

for i, row in tqdm(train_df.iterrows(), total=len(train_df)):
    try:
        emb = extract_whisper_embedding(row["file_path"])
        X.append(emb)
        y.append(row["label"])
    except Exception as e:
        print(f"Error processing {row['file']}: {e}")

100%|██████████| 444/444 [01:50<00:00,  4.01it/s]


In [272]:
X = np.vstack(X)
y = np.array(y)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [273]:
reg = MLPRegressor(hidden_layer_sizes=(512, 256), activation='relu',
                   solver='adam', max_iter=500, random_state=42)

reg.fit(X_train, y_train)
y_pred = reg.predict(X_val)

In [274]:
pearson_corr, _ = pearsonr(y_val, y_pred)
print("📈 Pearson Correlation:", round(pearson_corr, 4))

📈 Pearson Correlation: 0.7455


In [276]:
test_df = pd.read_csv("/kaggle/input/shl-dataset/dataset/test.csv")
test_df['file_path'] = test_df['filename'].apply(lambda x: os.path.join("/kaggle/input/shl-dataset/dataset/audios_test", x))

test_features = []

for file in tqdm(test_df["file_path"]):
    try:
        emb = extract_whisper_embedding(file)
        test_features.append(emb)
    except:
        test_features.append(np.zeros_like(X[0]))  # fallback

test_features = np.vstack(test_features)
test_preds = reg.predict(test_features)

100%|██████████| 195/195 [00:47<00:00,  4.07it/s]


In [277]:
import pandas as pd
import os

submission = pd.read_csv("/kaggle/input/shl-dataset/dataset/sample_submission.csv")

# Ensure your predictions (e.g., test_preds) have 195 entries
assert len(test_preds) == 195, f"Expected 195 predictions, but got {len(test_preds)}"

# Insert predictions
submission['label'] = test_preds

# Save to CSV (without index)
submission.to_csv("submission.csv", index=False)

# Confirm the file location
print("✅ Submission saved at:", os.path.abspath("submission.csv"))


df = pd.read_csv("submission.csv")
print(df.head())           # See first few rows
print(df.shape)            # Should print: (195, 2)

✅ Submission saved at: /kaggle/working/submission.csv
         filename     label
0   audio_706.wav  3.051158
1   audio_800.wav  2.875491
2    audio_68.wav  3.313574
3  audio_1267.wav  2.612978
4   audio_683.wav  2.115897
(195, 2)


In [278]:
download_file('/kaggle/working/submission.csv', 'out')


/kaggle/working/out.zip

In [280]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model("base").to(device)

/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
